In [10]:

import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [11]:
# Instantiate the data required in lists
users_list = ['ur117926588', 'ur3608743']
user_id = []
title_list = []
description_list = []
year_list = []
director_list = []
star_list = []
duration_list = []
genre_list = []
vote_list = []
movie_rating_list = []
user_rating_list = []
error_msg = []

In [12]:
for user in range(len(users_list)):
    driver = webdriver.Chrome('chromedriver.exe')
    url = 'https://www.imdb.com/user/{}/ratings'.format(users_list[user])
    time.sleep(1)
    driver.get(url)
    time.sleep(1)
    
    sel = Selector(text = driver.page_source)
    num_of_ratings = sel.css(".lister-list-length span::text").extract_first().replace(',','').split(' ')[0]
    rating_pages = int(int(num_of_ratings)/100) + 1  
    user_id += [users_list[user] for i in range(int(num_of_ratings))]

    for x in range(rating_pages):
        sel = Selector(text = driver.page_source)
        reviews = driver.find_elements(By.CSS_SELECTOR, 'div.lister-item-content')
        
        for review in tqdm(reviews):
            try:
                sel2 = Selector(text = review.get_attribute('innerHTML'))
                
                ## Get movie title
                try:
                    title = sel2.css('a::text').extract_first().strip()
                except:
                    title= np.NaN
                
                ## Get movie description
                try:
                    description = sel2.css('p::text').getall()[7].strip()
                except:
                    description = np.NaN
                ## Get movie year
                try:
                    year = sel2.css('.lister-item-year.text-muted.unbold::text').extract_first().strip().replace('(','').replace(')','')
                except:
                    year = np.NaN
                ## Get directors and staff
                try:
                    staff = sel2.css('.text-muted a::text').getall()
                    text = sel2.css('.text-muted.text-small::text').getall()
                    text2 = [x.strip() for x in text]
                    commas = text2.count(',')
                    stars_index = text2.index("Stars:")
                    count = 0
                    for i in range(stars_index, len(text2)-1):
                        if text2[i] == ',':
                            count+=1
                    stars = staff[-(count+1):]
                    directors = staff[:(commas-count)+1]
                except:
                    stars = np.NaN
                    directors = np.NaN
                
                ## Get movie duration
                try:
                    duration = sel2.css('.runtime::text').extract_first().strip()
                except:
                    duration = np.NaN
                ## Get Genre
                try:
                    genre = sel2.css('.genre::text').extract_first().strip()
                except:
                    genre = np.NaN
                ## Get votes
                try:
                    votes = sel2.css('.text-muted.text-small span::text').getall()[-1]
                except:
                    votes = np.NaN
                ## Get movie rating
                try:
                    movie_rating = sel2.css('.ipl-rating-star__rating::text').getall()[0].strip()
                except:
                    movie_rating = np.NaN
                ## Get user rating
                try:
                    user_rating = sel2.css('.ipl-rating-star__rating::text').getall()[1].strip()
                except:
                    user_rating = np.NaN
                
                title_list.append(title)
                description_list.append(description)
                year_list.append(year)
                director_list.append(directors)
                star_list.append(stars)
                duration_list.append(duration)
                genre_list.append(genre)
                movie_rating_list.append(movie_rating)
                user_rating_list.append(user_rating)
                
            except Exception as e:
                error_msg.append(e)
        try:
            next_page_url = sel.css("a.flat-button.lister-page-next.next-page::attr(href)").extract_first()
            driver.get("https://www.imdb.com" + next_page_url)
        except:
            print("No more pages to browse")

100%|██████████| 76/76 [00:01<00:00, 51.56it/s]


No more pages to browse


100%|██████████| 1/1 [00:00<00:00, 55.60it/s]

No more pages to browse


In [15]:
# Storing all data in dataframe
rating_df = pd.DataFrame({
     "UserID": user_id,
     "Title":title_list,
     "Description":description_list,
     "Year":year_list,
     "Directors":director_list,
     "Stars": star_list,
     "Duration": duration_list,
     "Genre": genre_list,
     "Movie_Rating": movie_rating_list,
     "User_Rating": user_rating_list})

In [16]:
# Export dataset
rating_df.to_csv(path_or_buf = "users_ratings.csv"
                          , index = False )